In [10]:
import os
os.chdir('../AiATrack')

from lib.test.evaluation import *
from collections import OrderedDict
import importlib
import cv2 as cv
import glob
import time

In [2]:
param_module = importlib.import_module('lib.test.parameter.aiatrack')
params = param_module.parameters('baseline')
tracker_module = importlib.import_module('lib.test.tracker.aiatrack')
tracker_class = tracker_module.get_tracker_class()

In [3]:
tracker = tracker_class(params, 'lasot', debug = True)


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
def _read_image(self, image_file: str):
    if isinstance(image_file, str):
        im = cv.imread(image_file)
        return cv.cvtColor(im, cv.COLOR_BGR2RGB)
    else:
        raise ValueError('ERROR: type of image_file should be str')

In [9]:
frames = sorted(glob.glob('/Users/aarsh/Desktop/CGVI/Thesis/Data/Test_3840x1920/0115/image/*.jpg'))
init_info = {
    'init_bbox': [2354.8, 1157.0, 26, 46]
}

In [ ]:
output = {'target_bbox': [],
                  'time': []}

if tracker.params.save_all_boxes:
    output['all_boxes'] = list()
    output['all_scores'] = list()
    

def _store_outputs(tracker_out: dict, defaults=None):
            defaults = dict() if defaults is None else defaults
            for key in output.keys():
                val = tracker_out.get(key, defaults.get(key, None))
                if key in tracker_out or val is not None:
                    output[key].append(val)

# Initialize
image = _read_image(frames[0])

start_time = time.time()
out = tracker.initialize(image, init_info)
if out is None:
    out = dict()

prev_output = OrderedDict(out)

init_default = {'target_bbox': init_info.get('init_bbox'),
                'time': time.time() - start_time}

if tracker.params.save_all_boxes:
    init_default['all_boxes'] = out['all_boxes']
    init_default['all_scores'] = out['all_scores']

_store_outputs(out, init_default)

for frame_num, frame_path in enumerate(frames[1:], start=1):
    image = _read_image(frame_path)

    start_time = time.time()

    out = tracker.track(image)

    prev_output = OrderedDict(out)
    _store_outputs(out, {'time': time.time() - start_time})